# **NaiveBayes Classifier**


## 1. Requirements

In [ ]:
# 한국어 전처리 라이브러리 
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 24.4 MB/s eta 0:00:00


In [ ]:
from tqdm import tqdm
from collections import defaultdict
import math

# POS(Part of Speech) tagger
from konlpy import tag 

#### 1.2 Train data & test data

In [ ]:
data = {}
# training data. input text 와 정답 label (긍정(1), 부정(0)) 로 구성.

data['train'] = [{'text': "정말 재미있습니다. 추천합니다."},
                {'text': "기대했던 것보단 별로였네요."},
                {'text': "지루해서 다시 보고 싶다는 생각이 안 드네요."},
                {'text': "완전 최고입니다 ! 다시 보고 싶습니다."},
                {'text': "연기도 연출도 다 만족스러웠습니다."},
                {'text': "연기가 좀 별로였습니다."},
                {'text': "연출도 좋았고 배우분들 연기도 최고입니다."},
                {'text': "기념일에 방문했는데 연기도 연출도 다 좋았습니다."},
                {'text': "전반적으로 지루했습니다. 저는 별로였네요."},
                {'text': "CG에 조금 더 신경 썼으면 좋겠습니다."}
                ]
# test data
data['test'] = [{'text': "최고입니다. 또 보고 싶네요."},
                {'text': "별로였습니다. 되도록 보지 마세요."},
                {'text': "다른 분들께 추천드릴 수 있을 만큼 연출도 연기도 만족했습니다."},
                {'text': "연기가 좀 더 개선되었으면 좋겠습니다."}
                ]

train_labels = [1, 0, 0, 1, 1, 0, 1, 1, 0, 0]
test_labels = [1, 0, 1, 0]

### 2. Data Preprocessing


#### 2.1 한글 형태소 분석기를 이용해서 주어진 데이터를 tokenize

[꼬꼬마(Kkma) 형태소 분석기](https://konlpy.org/en/v0.5.2/api/konlpy.tag/#module-konlpy.tag._kkma)를 이용

In [ ]:
# 형태소 분석기 선언
morph_analyzer = tag.Kkma() 

In [ ]:
# tokenization 함수 정의
def tokenization(data, morph_analyzer):
    '''
    (input) data: list of data examples.
            morph_analyzer: morphological analyzer.
    (output) tokenized_data: list of tokenized data examples.
    '''
    tokenized_data = []

    for example in tqdm(data):
        tokens = morph_analyzer.morphs(example['text'])
        tokenized_data.append(tokens)

    return tokenized_data

In [ ]:
# tokenization 함수를 이용한 데이터 tokenization
tokenized_data = {}

tokenized_data['train'] = tokenization(data['train'], morph_analyzer)
tokenized_data['test'] = tokenization(data['test'], morph_analyzer)

100%|██████████| 4/4 [00:00<00:00, 29.66it/s]


In [ ]:
# tokenized_data 확인
tokenized_data['train']

[['정말', '재미있', '습니다', '.', '추천', '하', 'ㅂ니다', '.'],
 ['기대', '하', '었', '더', 'ㄴ', '것', '보다', 'ㄴ', '별', '로', '이', '었', '네요', '.'],
 ['지루', '하', '어서', '다시', '보', '고', '싶', '다는', '생각', '이', '안', '들', '네요', '.'],
 ['완전', '최고', '이', 'ㅂ니다', '!', '다시', '보', '고', '싶', '습니다', '.'],
 ['연기', '도', '연출', '도', '다', '만족', '스럽', '었', '습니다', '.'],
 ['연기', '가', '좀', '별', '로', '이', '었', '습니다', '.'],
 ['연출', '도', '좋', '았', '고', '배우', '분', '들', '연기', '도', '최고', '이', 'ㅂ니다', '.'],
 ['기념일',
  '에',
  '방문',
  '하',
  '었',
  '는데',
  '연기',
  '도',
  '연출',
  '도',
  '다',
  '좋',
  '았',
  '습니다',
  '.'],
 ['전반적',
  '으로',
  '지루',
  '하',
  '었',
  '습니다',
  '.',
  '저',
  '는',
  '별',
  '로',
  '이',
  '었',
  '네요',
  '.'],
 ['CG', '에', '조금', '더', '신경', '쓰', '었', '으면', '좋', '겠', '습니다', '.']]

#### 2.2 tokenization 결과를 이용해서 word to index dictionary 를 생성


In [ ]:
# train data의 tokenization 결과에서 unique token만 남긴 set으로 변환
tokens = [token for i in range(len(tokenized_data['train'])) for token in tokenized_data['train'][i] ]
# unique token
unique_train_tokens = set(tokens)

# NaiveBayes Classifier의 input에 들어갈 word의 index를 반환해주는 dictionary를 생성
word2index = defaultdict() # key: word, value: index of word
idx = 0
for token in tqdm(unique_train_tokens):
    word2index[token] = idx
    idx += 1

100%|██████████| 56/56 [00:00<00:00, 482301.90it/s]


### 3. Model Training

#### 3.1 NaiveBayes Classifier 모델 클래스 구현


In [ ]:
class NaiveBayesClassifier():
    def __init__(self, word2index, k=0.1):
        """
        (input) word2index: mapping a word to a pre-assigned index.
        """
        self.k = k # for smoothing
        self.word2index = word2index
        self.priors = {} # Prior probability for each class, P(c)
        self.likelihoods = {} # Likelihood for each token, P(d|c)

    def _set_priors(self, labels):
        """
        Set prior probability for each class, P(c).
        Count the number of each class and calculate P(c) for each class.
        """
        
        class_counts = defaultdict(int)
        ############################ ANSWER HERE ################################
        # TODO 1: Count the number of each class
        # TODO 2: For each class, calcuate P(c)
        #########################################################################        
        for c in labels:
          class_counts[c]+=1
        for c, cnt in class_counts.items():
          self.priors[c] = class_counts[c]/len(labels)

    def _set_likelihoods(self, tokens, labels):
        """
        Set likelihood for each token, P(d|c).
        First, count the number of each class for each token.
        Then, calculate P(d|c) for a given class and token.
        """
        # distribute token
        token_dists = {}
        class_counts = defaultdict(int)

        for i, label in enumerate(tqdm(labels)):
            count = 0

            ############################ ANSWER HERE ################################
            # TODO: Count the number of each class for each token.
            #########################################################################

            # Tokenized Data vs word2Index
            for token in tokens[i]:
              if token not in token_dists:
                token_dists[token] = {0:0, 1:0}
              token_dists[token][label]+=1
            count += 1
            class_counts[label] += count

        # Chain Rule?
        for token, dist in tqdm(token_dists.items()):
            if token not in self.likelihoods:
                self.likelihoods[token] = {
                    0: (token_dists[token][0]+ self.k) / (class_counts[0] + len(self.word2index)* self.k),
                    1: (token_dists[token][1]+ self.k) / (class_counts[1] + len(self.word2index)* self.k),
                }

    def train(self, input_tokens, labels):
        """
        (input) input_tokens: list of tokenized train data.
                labels: train labels for each sentence/document.
        """
        self._set_priors(labels)
        self._set_likelihoods(input_tokens, labels)

    def inference(self, input_tokens):
        """
        (input) input_tokens: list_of tokenized test data.
        """
        log_prob_0 = 0.0
        log_prob_1 = 0.0

        # why not production / log -> summation?!
        for token in input_tokens:
            if token in self.likelihoods:
                log_prob_0 += math.log(self.likelihoods[token][0])
                log_prob_1 += math.log(self.likelihoods[token][1])

        log_prob_0 += math.log(self.priors[0])
        log_prob_1 += math.log(self.priors[1])

        if log_prob_0 >= log_prob_1:
            return 0
        else:
            return 1

#### 3.2 주어진 학습 데이터에 대해 문장 분류 모델 학습

In [ ]:
# 문장 분류 모델 선언 및 학습
# word2index { unique Token(word) : index } -> tokenized_data (Whole)
classifier = NaiveBayesClassifier(word2index)
classifier.train(tokenized_data['train'], train_labels)

100%|██████████| 56/56 [00:00<00:00, 306633.19it/s]


In [ ]:
result = classifier.likelihoods
result = sorted(result.items(), key=lambda x:x[0], reverse=False)
result

[('!', {0: 0.009433962264150943, 1: 0.10377358490566037}),
 ('.', {0: 0.5754716981132074, 1: 0.5754716981132074}),
 ('CG', {0: 0.10377358490566037, 1: 0.009433962264150943}),
 ('ㄴ', {0: 0.1981132075471698, 1: 0.009433962264150943}),
 ('ㅂ니다', {0: 0.009433962264150943, 1: 0.29245283018867924}),
 ('가', {0: 0.10377358490566037, 1: 0.009433962264150943}),
 ('것', {0: 0.10377358490566037, 1: 0.009433962264150943}),
 ('겠', {0: 0.10377358490566037, 1: 0.009433962264150943}),
 ('고', {0: 0.10377358490566037, 1: 0.1981132075471698}),
 ('기념일', {0: 0.009433962264150943, 1: 0.10377358490566037}),
 ('기대', {0: 0.10377358490566037, 1: 0.009433962264150943}),
 ('네요', {0: 0.29245283018867924, 1: 0.009433962264150943}),
 ('는', {0: 0.10377358490566037, 1: 0.009433962264150943}),
 ('는데', {0: 0.009433962264150943, 1: 0.10377358490566037}),
 ('다', {0: 0.009433962264150943, 1: 0.1981132075471698}),
 ('다는', {0: 0.10377358490566037, 1: 0.009433962264150943}),
 ('다시', {0: 0.10377358490566037, 1: 0.1037735849056603

### 4. Evaluation

각각의 Test 데이터에 대해 정답 추론 및 Accuracy score 계산

In [ ]:
# Test 데이터 inference
preds = []
for test_tokens in tqdm(tokenized_data['test']):
    pred = classifier.inference(test_tokens)
    preds.append(pred)

100%|██████████| 4/4 [00:00<00:00, 23629.88it/s]


In [ ]:
# Accuracy 측정
from sklearn.metrics import accuracy_score

print(accuracy_score(test_labels, preds))

1.0
